In [1]:
!pip install finterstellar

In [2]:
!pip install joblib joblib_progress tqdm

In [3]:
# Import library
import finterstellar as fs
#System trading Library 설치: Finterstellar

import io, sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
symbol_ = 'TSLA'
start_date_ = '2020-06-30'
end_date_ = '2025-06-30'
fee_ = 0.001

In [5]:
path_list = ["rsi","macd","stochastic"]

for path in path_list :
    try:
        os.makedirs(path, exist_ok=True)
        print(f"{path} 디렉토리가 생성되었거나 이미 존재합니다.")
    except OSError as e:
        print(f"디렉토리 생성 중 오류 발생: {e}")


rsi 디렉토리가 생성되었거나 이미 존재합니다.
macd 디렉토리가 생성되었거나 이미 존재합니다.
stochastic 디렉토리가 생성되었거나 이미 존재합니다.


In [6]:
# Performance 문자열을 Parsing 하여 반환하는 함수
def parse_performance_output(output_text):
    result = {}
    for line in output_text.strip().split("\n"):
        # 콜론으로 분리
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip()
        value = value.strip()

        # 퍼센트 처리
        if value.endswith("%"):
            num = float(value.strip("%")) / 100
            result[key] = num
        # 연도 처리
        elif value.endswith("yrs"):
            num = float(value.strip("yrs"))
            result[key] = num
        # 숫자 처리
        else:
            try:
                result[key] = float(value)
            except ValueError:
                result[key] = value  # 혹시 문자열이면 그대로 저장

    return result

In [7]:
# 1. RSI Performance and evaluation

In [8]:
# RSI 를 구해서 dict 를 반환하는 함수
def get_rsi_performance_dict(symbol_, df, params) :
    w_, buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol_
    result_dict["w_param"] = w_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # RSI 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.rsi(df_tmp, w=w_)
    fs.indicator_to_signal(df_tmp, factor='rsi', buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")

    return result_dict

In [9]:
# MACD 를 구해서 dict 를 반환하는 함수
def get_macd_performance_dict(symbol_, df, params) :
    buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # MACD 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.macd(df_tmp)
    fs.indicator_to_signal(df_tmp, factor='macd', buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")

    return result_dict

In [10]:
# Stochastic 를 구해서 dict 를 반환하는 함수
def get_stochastic_performance_dict(symbol_, df, params) :
    factor_, buy_, sell_, fee_ = params

    # dataFrame 초기화
    df_tmp = df.copy()

    result_dict = {}

    # 파라미터 추가
    result_dict["symbol"] = symbol_
    result_dict["factor"] = factor_
    result_dict["buy_param"] = buy_
    result_dict["sell_param"] = sell_
    result_dict["fee_"] = fee_

    perf = {}

    # Stochastic 계산, indicator signal 도출, position 계산, 평가 및 performance 측정
    fs.stochastic(df_tmp, symbol_, n=14, m=3, t=3)
    # indicator 값 추출을 위한 전처리 작업
    df_tmp['indicator'] = df_tmp['slow_k'] - df_tmp['slow_d']

    fs.indicator_to_signal(df_tmp, factor= factor_, buy=buy_, sell=sell_)
    fs.position(df_tmp)
    fs.evaluate(df_tmp, cost= fee_)

    # print 값 변수화
    capture = io.StringIO()
    # stdout 백업
    old_stdout = sys.stdout

    try:
        # stdout을 StringIO로 바꿈
        sys.stdout = capture

        # Performance 함수 실행 (print만 함)
        fs.performance(df_tmp, rf_rate=0.02)

    finally:
        # stdout 복원
        sys.stdout = old_stdout

    # 캡처된 텍스트
    output_text = capture.getvalue()
    # performance dict 화
    # performance_dict = parse_performance_output(output_text)
    perf = parse_performance_output(output_text)

    result_dict.update(perf)

    print(f"Performance Processing : {result_dict}")


    return result_dict

In [15]:
# 모든 팩터 집합을 이용해 최적의 RSI Parameter 계산
# Parallel 을 이용해서 성능 개선
from joblib import Parallel, delayed
from tqdm import tqdm

def get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    w_range = range(7,23+1)
    buy_range = range(15,45+1)
    sell_range = range(55,85+1)
    param_list = [(w,buy,sell,fee_) for w in w_range for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 w, buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for w in range(7,23+1)
    #   for buy in range(15, 45+1)
    #       for sell in range(55,85+1)
    #           results.append(get_rsi_performance_dict(df,w,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1, verbose=10, batch_size=100) (
            delayed(get_rsi_performance_dict)(symbol, df, params) for params in tqdm(param_list)
    )

    df_result = pd.DataFrame(results)
    return df_result

In [16]:
# 모든 팩터 집합을 이용해 최적의 MACD Parameter 계산
# Parallel 을 이용해서 성능 개선

def get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    buy_range = range(-10,10+1)
    sell_range = range(-10,10+1)
    param_list = [(buy,sell,fee_) for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for buy in range(-10,10+1)
    #    for sell in range(-10,10+1)
    #        results.append(get_macd_performance_dict(df,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1, verbose=10, batch_size=50) (
            delayed(get_macd_performance_dict)(symbol, df, params) for params in tqdm(param_list)

    )

    df_result = pd.DataFrame(results)
    return df_result

In [17]:
# 모든 팩터 집합을 이용해 최적의 Stochastic Parameter 계산
# Parallel 을 이용해서 성능 개선

def get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_) :

    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    print(f"[ {symbol} ] dataFrame.head ")
    print(df.head(3))

    # Parameter 집합 도출(list)
    factor_list = ['slow_k','indicator']
    buy_range = range(10,40+1)
    sell_range = range(60,90+1)
    param_list = [(factor_,buy,sell,fee_) for factor_ in factor_list for buy in buy_range for sell in sell_range]

    results = []

    # 최적의 buy_position, sell_position 을 찾기 위한 recursive
    # For문 이용
    # for factor in factor_list
    #   for buy in range(10, 40+1)
    #       for sell in range(60,90+1)
    #           results.append(get_stochastics_performance_dict(factor,buy,sell,fee_))

    # Parallel 이용
    results = Parallel(n_jobs=-1, verbose=10, batch_size=50) (
            delayed(get_stochastic_performance_dict)(symbol, df, params) for params in tqdm(param_list)
    )

    df_result = pd.DataFrame(results)
    return df_result

In [18]:
symbol_list = ['^GSPC', '^KS11','MSFT', 'GOOG', 'AMZN', 'META', 'NVDA', 'TSLA', 'AAPL']

In [19]:
# Parameter 조합을 csv 로 저장
for symbol in symbol_list :
    df_rsi = get_rsi_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_macd = get_macd_performance_dataframe(symbol, start_date_, end_date_, fee_)
    df_sto = get_stochastic_performance_dataframe(symbol, start_date_, end_date_, fee_)

    # write csv
    df_rsi.to_csv(f'./rsi/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_macd.to_csv(f'./macd/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)
    df_sto.to_csv(f'./stochastic/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

[ ^GSPC ] dataFrame.head 
              ^GSPC
2020-06-30 3,100.29
2020-07-01 3,115.86
2020-07-02 3,130.01


  0%|          | 0/16337 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   10.2s
  0%|          | 24/16337 [00:10<1:56:11,  2.34it/s][Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.5min
 22%|██▏       | 3624/16337 [10:04<34:56,  6.06it/s][Parallel(n_jobs=-1)]: Done 2624 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 3724 tasks      | elapsed: 13.4min
100%|██████████| 16337/16337 [42:52<00:00,  6.35it/s]
[Parallel(n_jobs=-1)]: Done 14964 tasks      | elapsed: 43.5min
[Parallel(n_jobs=-1)]: Done 15154 tasks      | elapsed: 44.0min
[Parallel(n_jobs=-1)]: Done 15734 tasks      | elapsed: 44.4min
[Parallel(n_jobs=-1)]: Done 15934 tasks      | elapsed: 44.7min
[Parallel(n_jobs=-1)]: Done 16049 tasks      | elapsed: 45.0min
[Parallel(n_jobs=-1)]: Done 16164 tasks      | elapsed: 45.3min
[Parallel(n_jobs=-1)]: Done 

[ ^GSPC ] dataFrame.head 
              ^GSPC
2020-06-30 3,100.29
2020-07-01 3,115.86
2020-07-02 3,130.01


  0%|          | 0/441 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
100%|██████████| 441/441 [00:00<00:00, 7977.54it/s]
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 207 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 246 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 291 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 336 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 387 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Don

[ ^GSPC ] dataFrame.head 
               Open     High      Low    Close      Volume  Adj Close
2020-06-30 3,050.20 3,111.51 3,047.83 3,100.29  4705850000   3,100.29
2020-07-01 3,105.92 3,128.44 3,101.17 3,115.86  4449230000   3,115.86
2020-07-02 3,143.64 3,165.81 3,124.52 3,130.01  4197720000   3,130.01


...

  0%|          | 0/16337 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.7s
  0%|          | 24/16337 [00:03<43:03,  6.31it/s][Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  3.3min
 22%|██▏       | 3624/16337 [09:41<34:01,  6.23it/s][Parallel(n_jobs=-1)]: Done 2624 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 3724 tasks      | elapsed: 13.0min
100%|██████████| 16337/16337 [41:49<00:00,  6.51it/s]
[Parallel(n_jobs=-1)]: Done 15059 tasks      | elapsed: 42.3min
[Parallel(n_jobs=-1)]: Done 15154 tasks      | elapsed: 42.8min
[Parallel(n_jobs=-1)]: Done 15734 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 15934 tasks      | elapsed: 43.5min
[Parallel(n_jobs=-1)]: Done 16049 tasks      | elapsed: 43.8min
[Parallel(n_jobs=-1)]: Done 16164 tasks      | elapsed: 44.0min
[Parallel(n_jobs=-1)]: Done 16

[ AAPL ] dataFrame.head 
            AAPL
2020-06-30 88.65
2020-07-01 88.49
2020-07-02 88.49


  0%|          | 0/441 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
100%|██████████| 441/441 [00:00<00:00, 8771.12it/s]
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 102 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 207 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 246 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 291 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 336 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 387 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Don

[ AAPL ] dataFrame.head 
            Open  High   Low  Close     Volume  Adj Close
2020-06-30 90.02 91.50 90.00  91.20  140223200      88.65
2020-07-01 91.28 91.84 90.98  91.03  110737200      88.49
2020-07-02 91.96 92.62 90.91  91.03  114041600      88.49


  0%|          | 0/1922 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    1.4s
100%|██████████| 1922/1922 [00:04<00:00, 432.60it/s]
[Parallel(n_jobs=-1)]: Done 1275 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1433 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1640 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 1653 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1715 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1828 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1843 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1860 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1922 out of 1922 | elapsed:    4.8s finished


In [ ]:
# Read_csv
# Max CAGR, Max Sharpe Ratio 값 구하기
# 해당 값에 해당하는 performance 값 계산 후 image 출력

In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_rsi_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_rsi = pd.read_csv(f'./rsi/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_rsi[df_rsi[text] == df_rsi[text].max()].iloc[0]
    max_w = max_series['w_param']
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # rsi
    fs.rsi(df, w=max_w)
    fs.draw_chart(df, left='rsi', right= symbol)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='rsi', buy=max_buy, sell=max_sell)
    # Calc position
    fs.position(df)
    fs.draw_chart(df, left='rsi', right='position_chart')
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # Evalution
    fs.evaluate(df, cost= fee_)
    fs.draw_chart(df, left='acc_rtn_dp', right=symbol)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_evaluation_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    # get Performance
    fs.performance(df, rf_rate=0.02)

    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./rsi/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_macd_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_macd = pd.read_csv(f'./macd/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_macd[df_macd[text] == df_macd[text].max()].iloc[0]
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_price(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_price_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # macd
    fs.macd(df)
    fs.draw_chart(df, right=['macd','macd_signal','macd_oscillator'])
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor='macd_oscillator', buy=max_buy, sell=max_sell)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='macd_oscillator')
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./macd/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [ ]:
# Max Parameter 를 이용하여 chart Graph 도출
def get_stochastic_max_performance(symbol, start_date_, end_date_, fee_, text) :
    df_sto = pd.read_csv(f'./stochastic/{symbol}_{start_date_}_to_{end_date_}.csv',encoding='utf8', index=False)

    max_series = df_sto[df_sto[text] == df_sto[text].max()].iloc[0]
    max_factor = max_series['factor']
    max_buy = max_series['buy_param']
    max_sell = max_series['sell_param']

    print(f"### {symbol}'s Parameter of max({text})")
    print(f"{max_series}")

    df = fs.get_ohlc(symbol, start_date=start_date_, end_date=end_date_)
    # df = pd.read_csv(f"get_ohlc_{symbol}_{start_date_}_to_{end_date_}.csv", encoding="utf8", index_col=0, parse_dates=True)

    # stochastic
    fs.stochastic(df, symbol, n=14, m=3, t=3)
    df['indicator'] = df['slow_k'] - df['slow_d']
    fs.draw_chart(df, left=max_factor, right=symbol)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_price_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

    fs.indicator_to_signal(df, factor=max_factor, buy=max_buy, sell=max_sell)

    # Calc position
    fs.position(df)
    fs.draw_chart(df, right='position_chart', left='slow_k')
    # # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_position_chart.png", dpi=300, bbox_inches="tight")
    # # 리소스 해제
    # plt.close()


    # Evalution
    fs.evaluate(df, cost=fee_)
    fs.performance(df, rf_rate=0.02)
    fs.draw_trade_results(df)
    # 파일로 저장
    plt.savefig(f"./stochastic/{symbol}_{text}_trade_result_chart.png", dpi=300, bbox_inches="tight")
    # 리소스 해제
    plt.close()

In [ ]:
cols = ['CAGR','Sharpe ratio']

for symbol in symbol_list :
    for col in cols :
        get_rsi_max_performance(symbol, start_date_, end_date_, fee_, col)
        get_macd_max_performance(symbol, start_date_, end_date_, fee_, col)
        get_stochastic_max_performance(symbol, start_date_, end_date_, fee_, col)